# <center>**Chapter 10 : Building Neural Nwtworks with Pytorch**</center>

## **Pytorch Fundamentals**



In [1]:
import torch


In [2]:
X = torch.tensor([[1.0 , 4.0 , 7.0] , [2.0 , 3.0 , 6.0]])
X

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [3]:
X.shape , X.dtype

(torch.Size([2, 3]), torch.float32)

In [4]:
# indexing works just like numpy arrays
X[0,1] , X[:,1]

(tensor(4.), tensor([4., 3.]))

In [5]:
import numpy as np
X.numpy()

array([[1., 4., 7.],
       [2., 3., 6.]], dtype=float32)

In [7]:
torch.tensor(np.array(np.array([[1 , 4 , 7] , [ 2 , 3 , 6]])))

tensor([[1, 4, 7],
        [2, 3, 6]])

In [8]:
#pytorch's API also provides many inplace operations

X.relu_()

tensor([[1., 4., 7.],
        [2., 3., 6.]])

## **Hardware Acceleration**


In [9]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [10]:
M = torch.tensor([[1. , 2. , 3.] , [4.,5.,6.]])
M = M.to(device)

In [11]:
M.device

device(type='cuda', index=0)

In [12]:
R = M @ M.T # this runs on the GPU
R

tensor([[14., 32.],
        [32., 77.]], device='cuda:0')

In [14]:
M = torch.rand(1000 , 1000) # on the CPU
%timeit M @ M.T

4.87 ms ± 206 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
M = torch.rand((1000 , 1000) , device = "cuda")
%timeit M @ M.T

436 μs ± 10.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## **Autograd**


In [18]:
x = torch.tensor(5.0 , requires_grad=True)
f = x ** 2
f

tensor(25., grad_fn=<PowBackward0>)

In [25]:
f.backward
x.grad

tensor(10.)

In [29]:
learning_rate = 0.1
with torch.no_grad():
    x -= learning_rate * x.grad  # this is the gradient decent step


In [30]:
# another way to avoid gradient computation
x_detached = x.detach()
x_detached -= learning_rate * x.grad

In [32]:
x.grad.zero_()

tensor(0.)